# Problem Statement
Implement and compare BFS, DFS with backtracking, and IDS (with DLS) for the N-Queens problem.
This notebook targets `N=8` for correctness checks and benchmarks `N in [4, 5, 6, 7, 8]`.


# Approach & State Representation
We represent a partial board as a list `state` where `state[row] = col`.
- `len(state)` is the number of assigned rows.
- One queen per row is guaranteed by construction.
- `isSafe(state, row, col)` enforces no shared column/diagonal.


In [ ]:
# If running in Colab and psutil is unavailable, uncomment:
# !pip -q install psutil

import importlib.util
import time
from collections import deque
from statistics import mean

psutil_spec = importlib.util.find_spec('psutil')
psutil = __import__('psutil') if psutil_spec is not None else None


# Algorithms Implemented
## Safety Check


In [ ]:
def isSafe(state, row, col):
    """Return True if placing a queen at (row, col) is conflict-free."""
    for r in range(row):
        c = state[r]
        if c == col:
            return False
        if abs(c - col) == abs(r - row):
            return False
    return True


In [ ]:
def measure_performance(func):
    """Decorator that returns (solution, elapsed_seconds, memory_kb_delta)."""
    def wrapper(n):
        process = psutil.Process() if psutil else None
        mem_before = process.memory_info().rss if process else 0
        start = time.time()
        solution = func(n)
        elapsed = time.time() - start
        mem_after = process.memory_info().rss if process else 0
        mem_kb = (mem_after - mem_before) / 1024
        return solution, elapsed, mem_kb
    return wrapper


## DFS Backtracking


In [ ]:
@measure_performance
def dfs_n_queens(n):
    """Solve N-Queens using DFS with backtracking; return first solution or None."""
    state = []

    def backtrack(row):
        if row == n:
            return state.copy()
        for col in range(n):
            if isSafe(state, row, col):
                state.append(col)
                result = backtrack(row + 1)
                if result is not None:
                    return result
                state.pop()
        return None

    return backtrack(0)


## BFS


In [ ]:
@measure_performance
def bfs_n_queens(n):
    """Solve N-Queens using BFS over partial states; return first solution or None."""
    queue = deque([[]])

    while queue:
        state = queue.popleft()
        row = len(state)
        if row == n:
            return state

        for col in range(n):
            if isSafe(state, row, col):
                queue.append(state + [col])

    return None


## IDS + DLS


In [ ]:
def dls(state, n, limit):
    """Depth-limited DFS. Does not expand beyond depth==limit."""
    row = len(state)

    if row == n:
        return state
    if row == limit:
        return None

    for col in range(n):
        if isSafe(state, row, col):
            result = dls(state + [col], n, limit)
            if result is not None:
                return result
    return None

@measure_performance
def ids_n_queens(n):
    """Iterative Deepening Search using DLS from depth limits 0..n."""
    for limit in range(n + 1):
        result = dls([], n, limit)
        if result is not None:
            return result
    return None


# Correctness Validation


In [ ]:
def is_valid_solution(sol, n):
    """Validate N-Queens solution format and constraints."""
    if sol is None or len(sol) != n:
        return False

    if any((not isinstance(c, int)) or c < 0 or c >= n for c in sol):
        return False

    for r1 in range(n):
        for r2 in range(r1 + 1, n):
            c1, c2 = sol[r1], sol[r2]
            if c1 == c2:
                return False
            if abs(c1 - c2) == abs(r1 - r2):
                return False
    return True


In [ ]:
for n in [4, 5, 8]:
    dfs_sol, _, _ = dfs_n_queens(n)
    bfs_sol, _, _ = bfs_n_queens(n)
    ids_sol, _, _ = ids_n_queens(n)

    print(f"N={n}")
    print(" DFS:", dfs_sol, "valid=", is_valid_solution(dfs_sol, n))
    print(" BFS:", bfs_sol, "valid=", is_valid_solution(bfs_sol, n))
    print(" IDS:", ids_sol, "valid=", is_valid_solution(ids_sol, n))
    print("-" * 50)


# Performance Measurement


In [ ]:
N_values = [4, 5, 6, 7, 8]
repeats = 3

results = {
    "DFS": {"time": [], "mem": []},
    "BFS": {"time": [], "mem": []},
    "IDS": {"time": [], "mem": []},
}

for n in N_values:
    dfs_times, dfs_mem = [], []
    bfs_times, bfs_mem = [], []
    ids_times, ids_mem = [], []

    for _ in range(repeats):
        _, t, m = dfs_n_queens(n)
        dfs_times.append(t)
        dfs_mem.append(m)

        _, t, m = bfs_n_queens(n)
        bfs_times.append(t)
        bfs_mem.append(m)

        _, t, m = ids_n_queens(n)
        ids_times.append(t)
        ids_mem.append(m)

    results["DFS"]["time"].append(mean(dfs_times))
    results["DFS"]["mem"].append(mean(dfs_mem))
    results["BFS"]["time"].append(mean(bfs_times))
    results["BFS"]["mem"].append(mean(bfs_mem))
    results["IDS"]["time"].append(mean(ids_times))
    results["IDS"]["mem"].append(mean(ids_mem))

results


# Plots


In [ ]:
import matplotlib.pyplot as plt

def plot_performance(n_values, results):
    """Plot runtime and memory trends for DFS, BFS, IDS."""
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    for alg in ["DFS", "BFS", "IDS"]:
        plt.plot(n_values, results[alg]["time"], marker='o', label=alg)
    plt.title("Runtime vs N")
    plt.xlabel("N")
    plt.ylabel("Time (seconds)")
    plt.xticks(n_values)
    plt.legend()

    plt.subplot(1, 2, 2)
    for alg in ["DFS", "BFS", "IDS"]:
        plt.plot(n_values, results[alg]["mem"], marker='o', label=alg)
    plt.title("Memory Delta (RSS) vs N")
    plt.xlabel("N")
    plt.ylabel("Memory change (KB)")
    plt.xticks(n_values)
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_performance(N_values, results)


# Discussion
- **DFS Backtracking**: Exponential worst-case, but strong pruning makes it fast for N up to 8.
- **BFS**: Also exponential, and typically uses much more memory because it stores broad frontiers.
- **IDS**: Memory usage is DFS-like, but it repeats shallower searches causing extra runtime overhead.

**Note on memory measurements:** RSS deltas can be noisy due to Python memory allocation/GC behavior.
